<a href="https://colab.research.google.com/github/xuhu357/DataAnalysis/blob/master/ch07_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A4%80%EB%B9%84_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

### Index Merge 

머지하려는 키가 DataFrame의 색인(index)일 수 있다. 

이때에는 left_index=True 혹은 right_index=True 옵션을 지정해 해당 색인을 머지 키로 사용할 수 있다.

In [0]:
left1 = DataFrame({
    'key': ['a', 'b', 'a', 'a', 'b', 'c'],
    'value': range(6)
})

right1 = DataFrame({
    'group_val': [3.5, 7]
}, index=['a', 'b'])

In [3]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [4]:
right1

,group_val
a,3.5
b,7.0


In [5]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


위에서 보다 싶이, merge는 기본적으로 inner join 즉 교집합을 구하지만, outer join을 통해서 합집합 처리 할 수 있다.

In [6]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


**계층적 색인**은 조금 복잡하다. 그러나, 천천히 하나하나씩 살펴보면 어렵지는 않다~

In [0]:
lefth = DataFrame({
    'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'key2': [2000, 2001, 2002, 2001, 2002], 
    'data': np.arange(5)
})

righth = DataFrame(
    np.arange(12).reshape((6, 2)),
    index = [
        ['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ],
    columns = ['event1', 'event2']
)

In [8]:
lefth

,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Nevada,2001
4,4,Nevada,2002


In [9]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

이럴 때에는 리스트로 여러 칼럼을 지정해서 머지해야 한다. 

중복되는 색인 값을 다룰때에는 주의해야 함. 

In [10]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1


발견 했겠지만, left_on의 길이와 right_index의 길이가 같아야 merge가 가능하다. 



---

outer merge방식으로 한번 해보자~

In [11]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


지금까지 본것은 left에는 column의 내용, 그리고 right에는 index 내용을 merge하는 과정이다. 

---

만약 양쪽에 공통으로 있는 여러개의 index를 merge한다면, 어떻게 할까? 좀 더 쉽다~

In [0]:
left2 = DataFrame(
    [[1., 2.], [3., 4.], [5., 6.]],
    index=['a', 'c', 'e'],
    columns=['Ohio', 'Nevada']
)

In [0]:
right2 = DataFrame(
    [
        [7., 8.], [9., 10.], [11., 12.], [13., 14.]
    ],
    index=['b', 'c', 'd', 'e'],
    columns=['Missouri', 'Alabama']
)

In [14]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [15]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


두 dataframe의 색인에 관해서 merge를 해보자. 방식은 outer

In [16]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


색인으로 머지 할 때, DataFrame의 join 메소드를 사용하면, 더 간편하다. 

join 메소드는 칼럼이 겹치지 않으며, 완전히 같거나 유사한 색인 구조를 가진 여러 개의 DataFrame 객체를 병합할 때 사용할 수 있다.


---

위의 예제를 고쳐쓰면, 

In [17]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [18]:
left2.join(right2) # outer로 지정해주지 않으면, default는 inner join이다.

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


DataFrame의 join 메소드는 ***왼쪽 조인***을 수행.

join메소드를 호출한 DataFrame의 칼럼중 하나에 대해서 조인을 수행하는것도 가능하다. 

left1의 key에 대해서 right1과 join을 해보자.

In [19]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [20]:
right1

,group_val
a,3.5
b,7.0


In [21]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


색인 대 색으로 두 DataFrame을 합치려면, 그냥 간단히 merge하려는 DataFrame의 리스트를 join 메소드에 넘기면 된다.

하지만, 일반적으로 이런 머지는 concat메소드를 사용한다~

In [0]:
another = DataFrame(
    [[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon']
)

left2에 right2 와 another DataFrame을 join해보자~

In [23]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [24]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### 축따라 이어붙이기

데이터를 합치는 또 다른 방법으로는 이어붙이기(concatenation), 연결(binding), 적층(stacking) 등이 있는데, Numpy는 ndarray를 연결하는 concatenate함수를 제공.

In [0]:
arr = np.arange(12).reshape((3, 4))

In [26]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [27]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

Series나 DataFrame 같은 pandas객체의 context내부에는 축마다 이름이 있어서,

배열을 쉽게 이어 붙일 수 있도록 되어 있다. 

* 만약에 연결하려는 두 객체의 색인이 서로 다르다면, 결과는 그 색인의 교집합이어야 하는가 아니면, 합집합이 어야 하는가?
* 합쳐진 결과에서 합쳐지기 전 객체의 데이터를 구분할 수 있는가?

* 어떤 축으로 연결할 것인지 고려해야 하는가?


concat함수는 위의 사항에 대한 해답을 제공한다. 다양한 예제를 통해서 하나씩 알아보자~

In [0]:
s1 = Series([0, 1], index=['a', 'b'])

s2 = Series([2, 3, 4], index=['c', 'd', 'e'])

s3 = Series([5, 6], index=['f', 'g'])

In [29]:
s1

a    0
b    1
dtype: int64

In [30]:
s2

c    2
d    3
e    4
dtype: int64

In [31]:
s3

f    5
g    6
dtype: int64

위의 s1, s2, s3에 대해서 concat 해보자~

In [32]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

concat함수는 보다 싶이 기본적으로 axis 0을 기본축으로 한다. 만약 axis=1로 지정한다면, 

series가 아닌 dataframe으로 변할것이다. 

In [33]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


겹치는 축이 없기 때문에 외부조인으로 정렬된 합집합을 얻었지만, join='inner'을 넘겨서 교집합을 구할 수도 있다.

In [34]:
pd.concat([s1, s2, s3], axis=1, join='inner')

,0,1,2


In [0]:
s4 = pd.concat([s1*5, s3])

In [36]:
s4

a    0
b    5
f    5
g    6
dtype: int64

In [37]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


join_axes 인자로 머지하려는 축을 직접 지정해줄수도 있다. 

In [38]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


Series를 이어 붙이기 전의 개별 Series를 구분할 수 없다는 문제가 생기는데, 이 경우 이어 붙이기 축에 대해 계층적 색인을 생성하여 식별이 가능하도록 할 수 있다. 

계층적 색인을 생성하려면 keys 인지를 사용하면 된다.

In [0]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])

In [40]:
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

제일 처음부분에 응용사례를 볼때, unstack함수를 보았었던 적이 있다. 

계층적 색인에 대해서 unstack을 일단 적용해보자~


In [43]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


후에 자세하게 설명하겠지만, 간략하게 설명하자면, unstack()하면, 계층적 index에서 제일 밖의 index를 column으로 풀어주는 역할을 한다. 
위의 예제에서 one, two, three는 level 0이고, a, b, c, d, e, f, g는 level 1이다. 


---

Series는 axis=1로 병합할 경우, keys는 DataFrame의 column이 된다. 

In [44]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


DataFrame의 객체에 대해서도 지금까지와 같은 방식으로 적용할 수 있다. 

In [0]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])

In [0]:
df2 = DataFrame(5+np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])

In [47]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

다른 방식으로 표현해보면, 리스트 대신 사전을 넘긴다면 사전의 키가 keys의 옵션으로 사용된다.

In [48]:
pd.concat({
    'level1': df1,
    'level2': df2
}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

알아보기 쉽게 이름을 추가해보자

In [49]:
pd.concat([df2, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1      level2     
lower  three four  three four
a          5    6      5    6
c          7    8      7    8

마지막으로, finally, DataFrame의 row index가 분석에 불필요한 경우에는 어떻게 할 것인가?

In [0]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [0]:
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [52]:
df1

,a,b,c,d
0,-0.735674,-0.187669,-1.361869,-2.316744
1,-0.576079,2.013463,0.197614,-2.357663
2,-0.312849,0.367638,0.061593,-0.257182


In [53]:
df2

,b,d,a
0,-0.827271,1.010536,-1.46199
1,1.010130,0.310346,-1.30075


row index가 분석에 불필요하다면, ignore_index 에 True 값을 줘서 무시하면 된다.

In [54]:
pd.concat([df1, df2], ignore_index=True) # axis 값을 명시하지 않았으므로 axis=0을 기본으로 합친다.

,a,b,c,d
0,-0.735674,-0.187669,-1.361869,-2.316744
1,-0.576079,2.013463,0.197614,-2.357663
2,-0.312849,0.367638,0.061593,-0.257182
3,-1.461990,-0.827271,NaN,1.010536
4,-1.300750,1.010130,NaN,0.310346


**concat 함수 인자 정리** 


* objs: 이어붙일 pandas 객체의 사전이나, 리스트, 필수 인자
* axis: 이어붙일 축의 방향, 기본 값은 0
* join: 조인 방식, 'inner' 방식도 있으며, 기본값은 'outer'
* join_axes: 합집합/교집합을 수행하는 대신 다른 n-1축으로 사용할 색인을 지정. 
* keys: 이어 붙일 객체나, 이어 붙인 축에 대한 계층 색인을 생성하는데, 연관 된 값이다. 리스트나 임의의 값이 들어있는 배열, 튜플의 배열 또는 배열의 리스트 (levels 옵션에 다차원 배열이 넘어온 경우)가 될수 있다. 
* levels: 계층 색인 레별로 사용할 색인을 지정한다. keys가 넘어온 경우 여러개의 색인을 지정.
* names: keys나 levels 혹은 둘다 있을 경우, 생성된 계층 레벨을 위한 이름
* verify_integrity: 이어 붙인 객체에 중복되는 축이 있는지 검사하고 있다면, 예외를 발생시킨다. 기본값은 False로 중복을 허용.
* ignore_index: 이어붙인 축의 색인을 유지하지 않고, range(total_length)로 새로운 색인을 생성한다. 